# Scienti API

In [1]:
import requests
import getpass
import pandas as pd
from IPython.display import JSON
import time
import json
import os
from varname import nameof

## Obtain products of "Apropiación Social del Conocimiento" (ASC) from Scienti API

### Read `"NIVEL_MAX"` for products of ASC

In [76]:
asc=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vSjpIpwy89MwRwJdXNy0_Z5zCK6XrieZ2_Cf7i1SQTuk4b_hRrNzuCTBgQe3eTOew/pub?output=xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [77]:
asc.columns

Index(['COD_RH', 'COD_PRODUCTO', 'SGL_CATEGORIA', 'NIVEL_0', 'NIVEL_1',
       'NIVEL_2', 'NIVEL_3', 'NIVEL_MAX', 'NÚMERO', 'CATEGORIAS', 'ASC', 'NC',
       'FRH', ' ', ' .1', ' .2', ' .3', ' .4', ' .5', ' .6', ' .7', ' .8',
       ' .9', ' .10', ' .11'],
      dtype='object')

In [4]:
asc[:1]

,COD_RH,COD_PRODUCTO,SGL_CATEGORIA,NIVEL_0,NIVEL_1,NIVEL_2,NIVEL_3,NIVEL_MAX,NÚMERO,CATEGORIAS,...,.2,.3,.4,.5,.6,.7,.8,.9,.10,.11
0,150.0,2.0,ART-00,1: Producción bibliográfica,11: Artículo,111: Publicado en revista especializada,NaN,111: Publicado en revista especializada,72886.0,"['ART-00', 'ART-ART', 'ART-GC', 'EPA-EPA', 'PE...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dasc=dict(asc[asc['ASC']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())

In [6]:
dnc=dict(asc[asc['NC']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())
dfrh=dict(asc[asc['FRH']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())

### Get all products ids from Scienti API

In [7]:
k=getpass.getpass('API KEY')

API KEY ········


Get full list of pro

In [8]:
institution='udea'
year=2023
p=requests.get(f"http://apis.colav.co/scienti/info/?apikey={k}&get=ids&institution={institution}&model_year={year}")

In [9]:
p.status_code

200

In [10]:
p=p.json()

In [11]:
pd.DataFrame(p[0].get('entities'))

,name,ids
0,network,"[{'COD_RH': '0000186473', 'COD_RED': 4}, {'COD..."
1,product,"[{'COD_RH': '0000000011', 'COD_PRODUCTO': '1'}..."
2,event,"[{'COD_RH': '0000000016', 'COD_EVENTO': 1}, {'..."
3,patent,"[{'COD_RH': '0000203190', 'COD_PATENTE': 1}, {..."
4,institution_endorsement,"[{'COD_AVAL_INSTITUCION': 1382}, {'COD_AVAL_IN..."
5,project,"[{'COD_RH': '0000000011', 'COD_PROYECTO': '1'}..."


In [12]:
p=[d for d in p[0].get('entities') if d.get('name')=='product']

In [13]:
len(p[0].get('ids'))

413913

### Filter products of ASC 

In [14]:
import json

In [18]:
def to_json(d,f):
    ob=open(f,'w')
    json.dump(d,ob)
    ob.close()
    
def read_json(f):
    ob=open(f,'r')
    d=json.load(ob)
    ob.close()
    return d

In [19]:
miss=read_json('missing.json')

In [21]:
try:
    if miss:
        pass
except:
    miss = p[0].get('ids')

In [22]:
len(miss)

238222

In [195]:
f"http://apis.colav.co/scienti/product?apikey={k}&model_year=2022&institution=udea&COD_RH={d.get('COD_RH')}&COD_PRODUCTO={d.get('COD_PRODUCTO')}"

'http://apis.colav.co/scienti/product?apikey=0colav.&model_year=2022&institution=udea&COD_RH=0000800503&COD_PRODUCTO=27'

In [28]:
ws=[]
wn=[]
wf=[]
ot=[]
ii=[]
i=110000
for d in miss[110000:]:
    print(f"{str(len(ws)).zfill(6)}:{str(len(wn)).zfill(6)}:{str(len(wf)).zfill(6)}:{str(len(ot)).zfill(6)}/{str(i).zfill(6)}",end='\r')    
    w=requests.get(f"http://apis.colav.co/scienti/product?apikey={k}&model_year=2022&institution=udea&COD_RH={d.get('COD_RH')}&COD_PRODUCTO={d.get('COD_PRODUCTO')}")
    if w.status_code!=200:
        continue
    if not w.json():
        continue    
    wj=w.json()
    x=wj.get('COD_TIPO_PRODUCTO')
    FOUND=False
    if isinstance(x,str) and x in dasc.keys():
        wj['TIPO_PRODUCTO']=dasc.get(x)
        ws.append(wj)
        FOUND=True
    if isinstance(x,str) and x in dnc.keys():
        wj['TIPO_PRODUCTO']=dnc.get(x)        
        wn.append(wj)
        FOUND=True        
    if isinstance(x,str) and x in dfrh.keys():
        wj['TIPO_PRODUCTO']=dfrh.get(x)        
        wf.append(wj)
        FOUND=True
    if not FOUND:
        ot.append(wj)
    
    time.sleep(0.01)
        
    if i%10000==0:
        to_json(ws,'asc4.json')
        to_json(wn,'nc4.json')
        to_json(wf,'frh4.json')
        to_json(ot,'other2.json')        
        to_json(ii,'i.json')
    #if len(wf)>0:
    #    break
    i+=1
    ii=[i]

        
to_json(ws,'asc4.json')
to_json(wn,'nc4.json')
to_json(wf,'frh4.json')        
to_json(ot,'other2.json')        

In [66]:
if w.json(): print(1)

In [23]:
cat i.json

[110000]

In [22]:
len(ws)

34877

In [16]:
import json

In [4]:
fp=open('asc.json','w')
json.dump(ws,fp)
fp.close()

NameError: name 'ws' is not defined

In [18]:
ls -lh asc.json

-rw-r--r-- 1 restrepo restrepo 263M Aug 22 20:59 asc.json


fp=open('asc.json','r')
ws=json.load(fp)
fp.close()

In [78]:
len(ws)

16881

### Export to table

In [83]:
#https://www.techieclues.com/blogs/how-to-get-variable-name-as-string-in-python
def name(variable):
    for name in globals():
        if id(globals()[name]) == id(variable):
            return name
    for name in locals():
        if id(locals()[name]) == id(variable):
            return name
    return None

def details(w):
    d=w.get('details')[0]
    if len(d.keys())==1:
        intd=d.get(list(d.keys())[0])
        if intd and isinstance(intd,list):
            newintd={}
            for intk in intd[0].keys():
                if intd[0][intk]:
                    newintd[intk]=intd[0][intk]
    return newintd

filterfields={
'general':['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO','TIPO_PRODUCTO', 'TXT_NME_PROD', 
       'TXT_RESUMEN_PROD', 
        'NRO_ANO_PRESENTA','NRO_MES_PRESENTA','DTA_CREACION','DTA_ACTUALIZACION',
        'TXT_WEB_PRODUCTO', 'SGL_CATEGORIA',
        'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR',  
       'NRO_AUTORES_TOTAL', 'NRO_GRUPOS',
       'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO'
       ],
'author':['TXT_TOTAL_NAMES','TXT_NAMES_RH',
'TXT_PRIM_APELL',
'TXT_SEG_APELL',
'TPO_NACIONALIDAD',
'TPO_DOCUMENTO_IDENT',
'TPO_SEXO',
'COD_ORCID',
'AUTOR_ID_SCP',
],
'group':['NRO_ID_GRUPO', 'COD_ID_GRUPO', 'NME_GRUPO','COD_RH_AREA', 'COD_AREA_CONHEC', 
       'COD_PROGRAMA', 'COD_PROGRAMA_SECUND','TXT_PLAN_TRABAJO', 
       'TXT_ESTADO_ARTE', 'TXT_OBJETIVOS', 'TXT_PROD_DESTACADA', 'TXT_RETOS', 'TXT_VISION',
       'area'],
'institution':['COD_INST', 'COD_INST_MACRO', 'NME_INST','SGL_INST', 
             'COD_SECTOR_ECON', 'SGL_PAIS', 'SGL_DEPARTAMENTO'],
'authors':['COD_RH_OTRO','TXT_TOTAL_NAMES_FILTRO'],
'project':['COD_RH', 'COD_PROYECTO', 'TXT_NME_PROYECTO', 'TXT_NME_PROYECTO_FILTRO', 'TXT_OBSERV_PROYECTO', 
         'TXT_RESUMEN_PROYECTO', 'TPO_PROYECTO', 'NRO_ANO_INICIO', 
         'NRO_ANO_FIN' 'NRO_DURACION', 'TPO_ESTADO_PROYECTO'],
'article':['TXT_VOLUMEN_REVISTA',
  'TXT_FASCICULO_REVISTA',
  'TXT_SERIE_REVISTA',
  'TXT_PAGINA_INICIAL',
  'TXT_PAGINA_FINAL'],
'journal':['TXT_ISSN_SEP','TXT_NME_REVISTA','SGL_PAIS','TPO_REVISTA','TPO_CLASIFICACION','TXT_URL'],   
'oriented_thesis': ['COD_PROGRAMA_ACADEMICO',
      'NRO_PAGINAS',
      'TPO_ORIENTACION',
      'NRO_MESES',
      'TPO_ESTADO',
      'TXT_NME_ORIENTADOS',
      'VALORACION_OBT_TESIS',
      'TPO_INSTITUCION',
      'NRO_ANO_FIN',
      'NRO_MES_FIN'],
'institution': [
        'NME_INST',
        'SGL_INST',
        'URL_HOME_PAGE',
        'TXT_CIUDAD_INST',
        'ID_INSTITUCION']    
}

In [80]:
f=open('asc3.json','r')
ws=json.load(f)
f.close()

In [81]:
len(ws)

68275

In [84]:
Ws=[]
for w in ws:#wf:#wn:
    W={}
    for k in filterfields['general']:
        W[k]=w.get(k)
    for sub in ['author','authors','group','institution','project','details']:
        if w.get(sub) and isinstance(w.get(sub),list):
            if sub != 'details':
                for kt in filterfields[sub]:
                    W[f'{sub}_{kt}']=w.get(sub)[0].get(kt)
            else:
                details=list(w.get(sub)[0].keys())[0]
                detailed=w.get(sub)[0].get(details)[0]
                if details in filterfields.keys():
                    for kt in filterfields[details]:
                        W[f'{details}_{kt}']=w.get(sub)[0].get(details)[0].get(kt)
                else:
                    W['details']=w.get(sub)[0].get(details)[0]
                try:
                    subdetails = [ x for x in detailed.keys() if isinstance(detailed.get(x),list) ][0]
                except:
                    subdetails = None
                    
                if subdetails and subdetails in filterfields.keys():
                    for kt in filterfields[subdetails]:
                        W[f'{subdetails}_{kt}']=w.get(sub)[0].get(details)[0].get(subdetails)[0].get(kt)                    
#                raise Exception('STOP')
    Ws.append(W)
    print(f'{str(len(Ws)).zfill(6)}',end='\r')
del ws#wf#wn

In [86]:
df=pd.DataFrame(Ws)

In [87]:
df.shape

(68275, 73)

In [100]:
df[60000:].to_excel('ASC6.xlsx',engine='xlsxwriter',index=None)
#df[:20000].to_excel('NC1.xlsx',index=None)
#df[20000:40000].to_excel('NC2.xlsx',index=None)
#df[40000:].to_excel('NC3.xlsx',index=None)
#df.to_excel('FTH3.xlsx',index=None)

In [54]:
del Ws

In [55]:
ls -lh FRH3.xlsx

-rw-r--r-- 1 restrepo restrepo 28M Sep 18 07:40 FRH3.xlsx


In [56]:
df[:1]

,COD_RH,COD_PRODUCTO,COD_TIPO_PRODUCTO,TIPO_PRODUCTO,TXT_NME_PROD,TXT_RESUMEN_PROD,NRO_ANO_PRESENTA,NRO_MES_PRESENTA,DTA_CREACION,DTA_ACTUALIZACION,...,oriented_thesis_COD_PROGRAMA_ACADEMICO,oriented_thesis_NRO_PAGINAS,oriented_thesis_TPO_ORIENTACION,oriented_thesis_NRO_MESES,oriented_thesis_TPO_ESTADO,oriented_thesis_TXT_NME_ORIENTADOS,oriented_thesis_VALORACION_OBT_TESIS,oriented_thesis_TPO_INSTITUCION,oriented_thesis_NRO_ANO_FIN,oriented_thesis_NRO_MES_FIN
0,0000483885,134,2J6,Ponencia,Determinantes genéticas de respuesta a la tera...,None,NaN,NaN,2019-05-29 14:30:47,2019-05-29 14:30:47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df.columns

Index(['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO', 'TIPO_PRODUCTO',
       'TXT_NME_PROD', 'TXT_RESUMEN_PROD', 'NRO_ANO_PRESENTA',
       'NRO_MES_PRESENTA', 'DTA_CREACION', 'DTA_ACTUALIZACION',
       'TXT_WEB_PRODUCTO', 'SGL_CATEGORIA', 'TXT_DOI', 'TXT_INST_PART',
       'DTA_EXPOSICION', 'STA_EVENTO', 'TXT_NME_TITULAR', 'NRO_AUTORES_TOTAL',
       'NRO_GRUPOS', 'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO',
       'author_TXT_TOTAL_NAMES', 'author_TXT_NAMES_RH',
       'author_TXT_PRIM_APELL', 'author_TXT_SEG_APELL',
       'author_TPO_NACIONALIDAD', 'author_TPO_DOCUMENTO_IDENT',
       'author_TPO_SEXO', 'author_COD_ORCID', 'author_AUTOR_ID_SCP', 'details',
       'group_NRO_ID_GRUPO', 'group_COD_ID_GRUPO', 'group_NME_GRUPO',
       'group_COD_RH_AREA', 'group_COD_AREA_CONHEC', 'group_COD_PROGRAMA',
       'group_COD_PROGRAMA_SECUND', 'group_TXT_PLAN_TRABAJO',
       'group_TXT_ESTADO_ARTE', 'group_TXT_OBJETIVOS',
       'group_TXT_PROD_DESTACADA', 'group_TXT_RETOS',

In [39]:
del df

In [38]:
from IPython.display import JSON

In [40]:
JSON(df.dropna(subset=['author_TXT_TOTAL_NAMES','group_NRO_ID_GRUPO','institution_NME_INST',
                 'project_TXT_NME_PROYECTO','TXT_RESUMEN_PROD']).iloc[0].to_dict())

<IPython.core.display.JSON object>

/usr/local/lib/python3.9/dist-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [32]:
import unidecode as ud

In [36]:
df.shape

(34877, 52)

In [41]:
df['TXT_NME_PROD'].str.lower().drop_duplicates().shape

(31262,)

In [35]:

df['TXT_NME_PROD'].str.lower().apply(ud.unidecode).drop_duplicates().shape

(31156,)

## Appendix

In [67]:
df=pd.DataFrame(ws)
df.columns

Index(['COD_RH', 'COD_PRODUCTO', 'TXT_NME_PROD', 'TXT_NME_PROD_FILTRO',
       'TXT_RESUMEN_PROD', 'TXT_OBSERV_PROD', 'SGL_IDIOMA',
       'COD_TIPO_PRODUCTO', 'COD_SUBTIPO_PRODUCTO', 'TXT_COMPLEMENTARIO',
       'COD_TIPO_PRODUCTO_MDCN', 'SGL_PAIS', 'NRO_ANO_PRESENTA',
       'NRO_MES_PRESENTA', 'TXT_WEB_PRODUCTO', 'NRO_IND_EXISTENCIA_MDCN',
       'NRO_IND_CALIDAD_MDCN', 'TXT_IND_CALIDAD_MDCN',
       'NRO_IND_VISIBILIDAD_MDCN', 'TPO_AMPERSAN_MDCN',
       'TPO_MEDIO_DIVULGACION', 'STA_RELEVANCIA', 'COD_PRODUCTO_BNR',
       'STA_CERTIFICADO', 'STA_VERIFICACION_AUTO', 'DTA_VERIF_AUTO',
       'STA_VERIFICACION_USUA', 'DTA_VERIF_USUA', 'TXT_FICHA_RESUMEN',
       'DTA_CREACION', 'DTA_ACTUALIZACION', 'STA_VENTANA', 'NRO_PATENTES',
       'NRO_SECRETOS', 'COD_UNO_MDCN', 'COD_DOS_MDCN', 'COD_TRES_MDCN',
       'STA_VENTANA_2', 'TXT_IND_CALIDAD_MDCN_2', 'TIPO_PRODUCTO_MDCN',
       'TIPO_PRODUCTO_MDCN_2', 'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR'

In [49]:
df['author'].iloc[0][0].keys()

dict_keys(['COD_RH', 'NRO_ID_CNPQ', 'TXT_NAMES_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES', 'TXT_NAMES_RH_FILTRO', 'TXT_PRIM_APELL_FILTRO', 'TXT_SEG_APELL_FILTRO', 'TXT_TOTAL_NAMES_FILTRO', 'TPO_NACIONALIDAD', 'DTA_NACIM', 'TPO_DOCUMENTO_IDENT', 'TPO_SEXO', 'COD_RH_MUN_NACIM', 'COD_MUN_NACIM', 'NRO_DOCUMENTO_IDENT', 'COD_RH_MUN_EXPED_DOC', 'COD_MUN_EXPED_DOC', 'NRO_LIBRETA_MILITAR', 'TPO_LIBRETA_MILITAR', 'NRO_PASAPORTE', 'NRO_VISA', 'TPO_VISA', 'DTA_INI_VIGENCIA_VISA', 'DTA_FIN_VIGENCIA_VISA', 'COD_RH_RELIGION', 'COD_RELIGION', 'SGL_PAIS_NACIM', 'TXT_AFICION', 'TXT_CONGREGA_PERTENECE', 'TXT_CONGREGA_PERTENECIO', 'NRO_CEDULA_EXTRANJ', 'TXT_COMENT_SUGEREN', 'DTA_DEFUNC', 'COD_NIVEL_FORMACION', 'TPO_ESTADO_CIVIL', 'TPO_DIRECCION_USUAL', 'TPO_DOC_ACTUALIZACION', 'TPO_VER_DOC_ACTUALIZACION', 'TXT_CITACION_BIBLIO', 'TXT_CITACION_BIBLIO_FILTRO', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'TXT_RESPUESTA', 'TXT_PREGUNTA', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'STA_

In [ ]:
author=['author_TXT_TOTAL_NAMES']

In [53]:
df['group'].dropna().iloc[0][0].keys()

dict_keys(['NRO_ID_GRUPO', 'COD_ID_GRUPO', 'NME_GRUPO', 'NME_GRUPO_FILTRO', 'ANO_FORMACAO', 'MES_FORMACAO', 'COD_RH_AREA', 'COD_AREA_CONHEC', 'COD_PROGRAMA', 'COD_PROGRAMA_SECUND', 'TPO_ESTADO_GR', 'DTA_ESTADO_GR', 'TXT_CLASIF', 'DTA_CLASIF', 'DTA_FIN_CLASIF', 'TPO_GRUPO', 'DTA_TPO_GRUPO', 'STA_ELIMINADO', 'TXT_PLAN_TRABAJO', 'TXT_ESTADO_ARTE', 'TXT_OBJETIVOS', 'TXT_PROD_DESTACADA', 'TXT_RETOS', 'TXT_VISION', 'TXT_SUGERENCIAS', 'XML_FICHA_RSM', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'COD_SESION', 'DTA_INI_SESION', 'TXT_CLASIF_1', 'COD_GRAN_AREA_CONHEC_MED', 'STA_CENTRO', 'TPO_CENTRO', 'STA_SECTOR', 'NRO_INTENTOS', 'STA_ACTIVO', 'COD_ACTIVACION', 'NRO_BLOQUEOS', 'COD_AREA_CONHEC_BK', 'institution', 'knowledge_area'])

In [ ]:
group=['group_NRO_ID_GRUPO', 'group_COD_ID_GRUPO', 'group_NME_GRUPO','group_COD_RH_AREA', 'group_COD_AREA_CONHEC', 
       'group_COD_PROGRAMA', 'group_COD_PROGRAMA_SECUND','group_TXT_PLAN_TRABAJO', 
       'group_TXT_ESTADO_ARTE', 'group_TXT_OBJETIVOS', 'group_TXT_PROD_DESTACADA', 'group_TXT_RETOS', 'group_TXT_VISION', 'group_TXT_SUGERENCIAS']


In [58]:
df['institution'].dropna().iloc[0][0].keys()

dict_keys(['COD_INST', 'COD_INST_MACRO', 'NME_INST', 'NME_INST_FILTRO', 'SGL_INST', 'COD_SECTOR_ECON', 'SGL_PAIS', 'SGL_DEPARTAMENTO', 'COD_RH_MUNICIPIO', 'COD_MUNICIPIO', 'URL_HOME_PAGE', 'TXT_DIRECCION', 'TXT_TELEFONO', 'TXT_FAX', 'TXT_EMAIL', 'TXT_NIT', 'TXT_DIGITO_VERIFICADOR', 'ID_REPRESENTANTE', 'TPO_AVAL', 'TXT_CIUDAD_INST', 'TPO_CARACTER', 'TPO_CONTRIBUYENTE', 'TXT_APARTADO_AEREO', 'STA_EXPORTA', 'COD_NIVEL', 'COD_IES', 'DTA_CONSTITUCION', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_SECTOR_ECON2', 'ID_INSTITUCION', 'STA_TERMINOS_COND', 'DTA_ACEPTA_TERMINOS'])

In [ ]:
institution=['institution_COD_INST', 'institution_COD_INST_MACRO', 'institution_NME_INST','institution_SGL_INST', 'institution_COD_SECTOR_ECON', 'institution_SGL_PAIS', 'institution_SGL_DEPARTAMENTO']

In [59]:
df['author_others'].dropna().iloc[0][0].keys()

dict_keys(['COD_RH_CREA', 'COD_RH_OTRO', 'TXT_NME_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES_FILTRO', 'SGL_PAIS', 'TPO_NACIONALIDAD', 'DTA_NACIMIENTO', 'TPO_DOC_IDENTIFICACION', 'NRO_DOC_IDENTIFICACION', 'NRO_ID_CNPQ', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'NRO_DOC_FILTRO', 'TXT_TOTAL_NAMES', 'TXT_NME_RH_FILTRO', 'TXT_PRIM_APELL_FILTRO', 'TXT_SEG_APELL_FILTRO', 'COD_RH_REF', 'COD_ORCID', 'AUTOR_ID_SCP'])

In [ ]:
authors=['authors_COD_RH_OTRO','authors_TXT_TOTAL_NAMES_FILTRO']

In [72]:
df['project'].dropna().iloc[0][0].keys()

dict_keys(['COD_RH', 'COD_PROYECTO', 'TXT_NME_PROYECTO', 'TXT_NME_PROYECTO_FILTRO', 'TXT_OBSERV_PROYECTO', 'TXT_RESUMEN_PROYECTO', 'TPO_PROYECTO', 'NRO_MES_INICIO', 'NRO_ANO_INICIO', 'NRO_MES_FIN', 'NRO_ANO_FIN', 'NRO_MES_PROYEC', 'NRO_ANO_PROYEC', 'NRO_DURACION', 'TPO_ESTADO_PROYECTO', 'NRO_GRADUACION', 'NRO_ESPECIALIZACION', 'NRO_MAESTRIA_ACAD', 'NRO_MAESTRIA_PROF', 'NRO_DOCTORADO', 'COD_PROYECTO_BNR', 'STA_CERTIFICADO', 'STA_VERIFICACION_AUTO', 'DTA_VERIF_AUTO', 'STA_VERIFICACION_USUA', 'DTA_VERIF_USUA', 'TXT_FICHA_RESUMEN', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'TXT_URL', 'COD_PROYECTO_SNCTI', 'TPO_FINANCIACION', 'TPO_FTE_FINAN', 'TPO_AMB_FINAN', 'STA_AVAL_INST', 'DTA_AVAL_INST', 'TPO_AVAL_INST', 'COD_INST_AVALA', 'ID_USUARIO_AVALA', 'DTA_ACTUALIZACION_CVLAC', 'CAT_MED', 'SGL_CATEGORIA'])

In [ ]:
project=['project_COD_RH', 'project_COD_PROYECTO', 'project_TXT_NME_PROYECTO', 'project_TXT_NME_PROYECTO_FILTRO', 'project_TXT_OBSERV_PROYECTO', 
         'project_TXT_RESUMEN_PROYECTO', 'project_TPO_PROYECTO', 'project_NRO_ANO_INICIO', 
         'project_NRO_ANO_FIN' 'project_NRO_DURACION', 'project_TPO_ESTADO_PROYECTO']

In [68]:
df['details'].dropna()

0        [{'event': [{'COD_RH': '0000483885', 'COD_EVEN...
1        [{'event': [{'COD_RH': '0000084328', 'COD_EVEN...
2        [{'event': [{'COD_RH': '0000101443', 'COD_EVEN...
3        [{'event': [{'COD_RH': '0001478679', 'COD_EVEN...
4        [{'keywords': [{'COD_RH': '0000118139', 'COD_P...
                               ...                        
71469    [{'event': [{'COD_RH': '0001347073', 'COD_EVEN...
71470    [{'memory_chapter': [{'COD_RH': '0000047945', ...
71471    [{'event': [{'COD_RH': '0001338966', 'COD_EVEN...
71473    [{'book_chapter': [{'COD_RH': '0000031291', 'C...
71475    [{'memory_chapter': [{'COD_RH': '0000217042', ...
Name: details, Length: 57850, dtype: object

In [ ]:
df.shape

(71476, 93)

In [58]:
df=df.drop_duplicates(subset=['COD_TIPO_PRODUCTO']).dropna(subset='details').reset_index(drop=True)

In [71]:
df.details

0     [{'event': [{'COD_RH': '0000483885', 'COD_EVEN...
1     [{'event': [{'COD_RH': '0000101443', 'COD_EVEN...
2     [{'keywords': [{'COD_RH': '0000118139', 'COD_P...
3     [{'memory_chapter': [{'COD_RH': '0000296635', ...
4     [{'technical': [{'COD_RH': '0000199761', 'COD_...
5     [{'memory_chapter': [{'COD_RH': '0000087661', ...
6     [{'application_sector': [{'COD_SECTOR_APLICACI...
7     [{'book_chapter': [{'COD_RH': '0000902004', 'C...
8     [{'technical': [{'COD_RH': '0000328294', 'COD_...
9     [{'technical': [{'COD_RH': '0000056834', 'COD_...
10    [{'technical': [{'COD_RH': '0000260193', 'COD_...
11    [{'article': [{'COD_RH': '0000827851', 'COD_PR...
12    [{'event': [{'COD_RH': '0000035889', 'COD_EVEN...
13    [{'book': [{'COD_RH': '0000616028', 'COD_PRODU...
14    [{'book': [{'COD_RH': '0000218073', 'COD_PRODU...
15    [{'book': [{'COD_RH': '0000175226', 'COD_PRODU...
16    [{'technical': [{'COD_RH': '0000156612', 'COD_...
17    [{'event': [{'COD_RH': '0000150126', 'COD_

In [26]:
df.details.str[0].str['event'].dropna().iloc[0]

[{'COD_RH': '0001127420',
  'COD_EVENTO': 14,
  'TPO_EVENTO': 'EN',
  'TXT_NME_EVENTO': 'XI Encuentro Regional Uceva de actores en seguridad Alimentaria, Cuidado de la Salud y del Ambiente',
  'COD_RH_MUNICIPIO': '0000000000',
  'COD_MUNICIPIO': '1252',
  'NRO_ANO_REALIZACION': None,
  'STA_ITINERANTE': 'F',
  'STA_CATALOGO': 'F',
  'NRO_DURACION': None,
  'TXT_LUGAR': 'Uceva',
  'TPO_CLASIFICACION': 'N',
  'TPO_INSTITUCION': 'CV',
  'COD_INST_FINANCIA': '1013',
  'COD_INST': None,
  'COD_INST_OTRO': None,
  'DTA_CREACION': '2018-11-23 11:14:27',
  'DTA_ACTUALIZACION': '2018-11-23 11:14:27',
  'TPO_PARTICIPACION': None,
  'TPO_LUGAR': None,
  'NRO_PARTICIPANTES': None,
  'TXT_RESUMEN_EVENTO': None,
  'TXT_URL': None,
  'TXT_CRONOGRAMA': None,
  'TXT_ACTIVIDADES': None,
  'CLASE_EVENTO': 'EI',
  'DTA_INICIO': '2018-11-02 00:00:00',
  'DTA_FIN': '2018-11-02 00:00:00',
  'TXT_NME_EVENTO_FILTRO': None,
  'TMP_EVE_PA': None,
  'DTA_ACTUALIZACION_CVLAC': None,
  'DTA_AVAL_INST': None,
  'TPO

In [33]:
df.details.str[0].str['technical'].dropna().iloc[5]

[{'COD_RH': '0001135651',
  'COD_PRODUCTO': '1',
  'TXT_CONTRATO_REG': None,
  'TXT_CERT_INSTITUCION': 'Universidad de Antioquia',
  'TXT_NME_COMERCIAL': 'Escuela Digital - Las Funciones del Lenguaje',
  'TXT_DISPONIBILIDAD': 'Restringido',
  'TXT_FINALIDAD': 'Material audiovisual interactivo sobre las funciones del lenguaje',
  'COD_RH_MUNICIPIO': '0000000000',
  'COD_MUNICIPIO': None,
  'TPO_INSTITUCION': 'IL',
  'COD_INSTITUCION': '007300000887',
  'COD_INST': '007300000887',
  'COD_INST_OTRO': None,
  'DTA_CREACION': '2010-05-18 10:29:18',
  'DTA_ACTUALIZACION': '2013-10-26 16:15:29',
  'TPO_TAM_EMP_GR': 'F',
  'TPO_TAM_EMP_MP': 'F',
  'DTA_ENVIO': None,
  'ID_INSTITUCION_EMITE': None,
  'ID_INSTITUCION': 883,
  'ANO_OBTENCION': None,
  'MES_OBTENCION': None,
  'tech_product': [{'COD_RH': '0001135651',
    'COD_PRODUCTO': '1',
    'TXT_PLATAFORMA': None,
    'TXT_AMBIENTE': None,
    'TXT_TEMA': None,
    'TXT_TECNICA': None,
    'TXT_AREA_REPRESENTA': None,
    'COD_EDITORIAL': No

In [34]:
df.details.str[0].str['article'].dropna().iloc[0]

[{'COD_RH': '0001127217',
  'COD_PRODUCTO': '224',
  'COD_REVISTA': 4583,
  'COD_REVISTA_OTRO': None,
  'COD_RH_MUNICIPIO': '0000000000',
  'COD_MUNICIPIO': '249',
  'TXT_VOLUMEN_REVISTA': '30',
  'TXT_FASCICULO_REVISTA': None,
  'TXT_SERIE_REVISTA': None,
  'TXT_PAGINA_INICIAL': '110',
  'TXT_PAGINA_FINAL': '114',
  'DTA_PUBLICACION': None,
  'DTA_CREACION': '2021-05-07 11:19:01',
  'DTA_ACTUALIZACION': '2021-05-07 11:19:01',
  'TXT_URL_CAT_EDITORIAL': None,
  'TXT_URL_POL_EDITORIAL': None,
  'MED_REVISION': None,
  'journal': [{'COD_REVISTA': 4583,
    'TXT_ISSN_SEP': '0123-3386',
    'TXT_ISSN': '01233386',
    'COD_REVISTA_REF': None,
    'TXT_ISSN_REF_SEP': None,
    'TXT_ISSN_REF': None,
    'TXT_NME_REVISTA': 'Agoras. Anuario De La Asociación De Profesores De La Universidad De Antioquia',
    'TXT_NME_REVISTA_FILTRO': 'AGORAS ANUARIO DE LA ASOCIACION DE PROFESORES DE LA UNIVERSIDAD DE ANTIOQUIA',
    'SGL_PAIS': 'COL',
    'TPO_REVISTA': 'P',
    'CLASE': None,
    'TPO_CLASIFIC

In [36]:
df.details.str[0].str['audiovisual'].dropna().iloc[1]

[{'COD_RH': '0001201662',
  'COD_PRODUCTO': '114',
  'TXT_EMISORA': 'Emisora Cultura Universidad de Antioquia',
  'DTA_EMISION': '2020-05-25 00:00:00',
  'NRO_DURACION': 5,
  'COD_RH_MUNICIPIO': '0000000000',
  'COD_MUNICIPIO': '249',
  'TXT_FINALIDAD': 'N',
  'DTA_CREACION': '2020-07-12 02:15:40',
  'DTA_ACTUALIZACION': '2020-07-12 02:15:40',
  'city': [{'COD_RH_MUNICIPIO': '0000000000',
    'COD_MUNICIPIO': '249',
    'ID_MUNICIPIO': 385,
    'SGL_DEPARTAMENTO': 'AN',
    'SGL_PAIS': 'COL',
    'COD_MUNI_INTERNACIONAL': None,
    'COD_MUNI_DANE': '05001',
    'TXT_NME_MUNICIPIO': 'MEDELLÍN',
    'TXT_NME_MUNICIPIO_FILTRO': 'MEDELLIN',
    'ID_PERSONA': None,
    'DTA_CREACION': '2008-05-07 12:29:11',
    'DTA_ACTUALIZACION': '2022-04-19 11:04:53',
    'ID_PAIS': 1,
    'ID_DEPARTAMENTO': 31,
    'ID': None,
    'department': [{'SGL_PAIS': 'COL',
      'SGL_DEPARTAMENTO': 'AN',
      'ID_DEPARTAMENTO': 31,
      'SGL_REGION': 'NW',
      'COD_DEPA_DANE': '05',
      'TXT_NME_DEPARTAME

In [74]:
df.details.str[0].str['memory_chapter'].dropna().iloc[1]#,'book_chapter','book'

[{'COD_RH': '0000087661',
  'COD_PRODUCTO': '30368',
  'TXT_VOLUMEN': None,
  'TXT_FASCICULO': None,
  'TXT_SERIE': None,
  'TXT_PAGINA_INICIAL': None,
  'TXT_PAGINA_FINAL': None,
  'COD_LIBRO_REF': None,
  'COD_LIBRO_OTRO': None,
  'TXT_NME_PONENCIA': 'ASOCIACIÓN DE VARIANTES ALÉLICAS EN GENES RELACIONADOS CON LAACTIVIDAD FUNCIONAL DE LA VITAMINA D CON LA RESISTENCIA NATURAL A LA INFECCIÓN POR EL VIH-1',
  'TXT_NME_EVENTO': 'II simposio de inmunomodulación y III encuentro nacional de investigadores en inmunología',
  'TPO_EVENTO': 'N',
  'NRO_ANO_EVENTO': 2012,
  'TXT_LUGAR': None,
  'COD_RH_MUNICIPIO': '0000000000',
  'COD_MUNICIPIO': '249',
  'TXT_NME_INSTITUCION': None,
  'DTA_CREACION': '2012-10-18 13:07:48',
  'DTA_ACTUALIZACION': '2012-10-18 13:14:22',
  'TXT_NME_EVENTO_FILTRO': 'II SIMPOSIO DE INMUNOMODULACION Y III ENCUENTRO NACIONAL DE INVESTIGADORES EN INMUNOLOGIA',
  'COD_REVISTA': None,
  'COD_REVISTA_OTRO': None,
  'city': [{'COD_RH_MUNICIPIO': '0000000000',
    'COD_MUNI

In [76]:
df.details.str[0].str['book_chapter'].dropna().iloc[0]#,,'book'

[{'COD_RH': '0000902004',
  'COD_PRODUCTO': '18',
  'NRO_PAGINAS': 449,
  'TXT_VOLUMEN': None,
  'TXT_SERIE': '1',
  'TXT_EDICION': '1',
  'TXT_PAGINA_INICIAL': '199',
  'TXT_PAGINA_FINAL': '215',
  'COD_LIBRO_REF': None,
  'COD_LIBRO_OTRO': '6',
  'TXT_ORGANIZADORES': 'Luis Fernando Garcés Giraldo',
  'DTA_CREACION': '2011-08-19 12:00:46',
  'DTA_ACTUALIZACION': '2014-09-08 15:23:51',
  'TXT_URL_CAT_EDITORIAL': None,
  'TXT_URL_POL_EDITORIAL': None,
  'book_other': [{'COD_RH': '0000902004',
    'COD_PRODUCTO': '6',
    'TXT_NME_LIBRO': 'Serie Libro Lasallista Investigación Y Ciencia',
    'TXT_NME_LIBRO_FILTRO': 'SERIE LIBRO LASALLISTA INVESTIGACION Y CIENCIA',
    'TXT_ISBN': '978-958-8406-12-1',
    'TXT_ISBN_F': '9789588406121',
    'NRO_VOLUMENES_LIBRO': 200,
    'TXT_NRO_PAGINAS': '449',
    'TXT_EDICION': '1',
    'TXT_SERIE_LIBRO': 'Lasallista Investigación Y Ciencia',
    'SGL_IDIOMA': 'ES',
    'TXT_AUTORES': 'Luis Fernando Garcés Giraldo',
    'COD_EDITORIAL': 12539,
    'CO

In [81]:
df.details.str[0].str['book'].dropna().iloc[2]#,,

[{'COD_RH': '0000175226',
  'COD_PRODUCTO': '74',
  'TXT_NME_LIBRO': 'Origen y evolución de la vida',
  'TXT_NME_LIBRO_FILTRO': 'ORIGEN Y EVOLUCION DE LA VIDA',
  'TXT_ISBN': '978-958-5518-08-7',
  'TXT_ISBN_F': '9789585518087',
  'NRO_VOLUMENES_LIBRO': None,
  'TXT_NRO_PAGINAS': None,
  'TXT_EDICION': None,
  'TXT_SERIE_LIBRO': None,
  'TXT_REF_LIBRO': None,
  'SGL_IDIOMA': None,
  'COD_EDITORIAL': None,
  'COD_EDITORIAL_OTRO': 8,
  'NRO_ES_VALIDO': 1,
  'DTA_CREACION': '2019-01-21 15:03:24',
  'DTA_ACTUALIZACION': '2019-01-21 15:03:24',
  'TXT_URL_CAT_EDITORIAL': None,
  'TXT_URL_POL_EDITORIAL': None,
  'TPO_PUBLICACION': None,
  'COD_CENTRO_INV': None,
  'TXT_ISSN': None,
  'TPO_LIBRO': None,
  'editorial_others': [{'COD_RH': '0000175226',
    'COD_EDITORIAL': 8,
    'TXT_NME_EDITORIAL': 'Fondo Editorial Universidad Católica de Oriente',
    'TXT_NME_EDITORIAL_FILTRO': 'FONDO EDITORIAL UNIVERSIDAD CATOLICA DE ORIENTE',
    'COD_RH_MUNICIPIO': '0000000000',
    'COD_MUNICIPIO': '333'

In [ ]:
JSON(wn[0])

<IPython.core.display.JSON object>

Missing

In [169]:
file='/home/restrepo/Downloads/asc2.json'
f=open(file,'r')
wn=json.load(f)
f.close()

In [170]:
f'rm {file}'

'rm /home/restrepo/Downloads/asc2.json'

In [171]:
if file and file.find('/')>-1:
    os.system(f'rm {file}')

In [172]:
len(miss)

248946

In [173]:
wn= [{'COD_RH':d.get('COD_RH'),'COD_PRODUCTO':d.get('COD_PRODUCTO')} for d in wn]

In [174]:
wn[:3]

[{'COD_RH': '0001127217', 'COD_PRODUCTO': '103'},
 {'COD_RH': '0001127217', 'COD_PRODUCTO': '104'},
 {'COD_RH': '0001127217', 'COD_PRODUCTO': '123'}]

In [175]:
if miss:
    p=miss
else:
    p=[d for d in p[0].get('ids')]

In [176]:
len(p)

248946

In [177]:
miss=set([ str(d) for d in p]).difference( [str(d) for d in wn]  )

In [178]:
miss=[eval(d) for d in miss]

In [179]:
len(miss)

238222

In [180]:
f=open('missing.json','w')
json.dump(miss,f)
f.close()

In [2]:
DOI_Sostenibilidad='https://docs.google.com/spreadsheets/d/e/2PACX-1vS-LMVMYusTHZPI5Bt-c7WfA6pPq6Ayf1FruWHcVThXXHzspSYqDKI5c30AjjoWZg/pub?output=xlsx'

In [64]:
dois=pd.read_excel(DOI_Sostenibilidad)

In [65]:
dois.shape

(2335, 1)

In [ ]:
dois['DOI']=dois.Norm_DOI.str.replace('https*\:\/\/[\w.]+/','').str.lower()

In [59]:
def check_affiliation(d,ror='03bp5hc83'):
    found=False
    p=requests.get(f'https://api.openalex.org/works/https://doi.org/{d}')
    if (p.status_code == 200 and
        str(p.json().get('authorships')).find(f'https://ror.org/{ror}')>-1
       ):
        found=True
        
    time.sleep(1/10)    
    return found

In [61]:
dois['check']=dois.DOI.apply(check_affiliation)

In [68]:
dois[dois['check']].shape

(1761, 3)

In [ ]:
dois.to_excel(f'{nameof(DOI_Sostenibilidad)}.xlsx',index=False)

In [4]:
dois=pd.read_excel(f'{nameof(DOI_Sostenibilidad)}.xlsx')

In [53]:
def check_affiliation_hep(d,grid='grid.412881.6'):
    found=d.get('check')
    if not d.get('check'):
        p=requests.get(f"https://inspirehep.net/api/literature?q=doi:{d.get('DOI')}")
        if p.status_code == 200 and p.json().get('hits').get('hits'):
            if (L and 
                isinstance(L,list) and 
                str(L[0].get('metadata').get('authors')).find('grid.412881.6') > -1
               ):
                found=True
        
        time.sleep(1/10)    
    return found

In [ ]:
dois=dois.apply(lambda d: check_affiliation_hep(d),axis='columns')

In [43]:
d={'DOI':'10.1007/JHEP08(2023)040ff'}

In [44]:
p=requests.get(f"https://inspirehep.net/api/literature?q=doi:{d.get('DOI')}")

In [45]:
L=p.json().get('hits').get('hits')
if L and isinstance(L,list):
    L=L[0]

In [46]:
L

[]

In [22]:
f"https://inspirehep.net/literature?q=doi:{d.get('DOI')}"

'https://inspirehep.net/literature?q=doi:10.1016/j.nima.2023.168452'

In [41]:
str(L[0].get('metadata').get('authors')).find('grid.412881.6') > -1

True